In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import numpy as np

In [ ]:
!pip freeze > requirements.txt

In [ ]:
import pandas as pd

df1 = pd.read_csv("/content/DataSetQ&A2.csv",encoding="Windows-1252",sep=';')
df2 = pd.read_csv("/content/Classeur1.csv",encoding="Windows-1252",sep=';')
df3 = pd.read_csv("/content/AugmentedQ&A.csv")
#df4 = pd.read_csv("/content/batch_2_a6 (1).csv")
#df5 = pd.read_csv("/content/batch_2_a6.csv")
#df6 = pd.read_csv("/content/batch_2_a8.csv")
#df7 = pd.read_csv("/content/batch_3_a7.csv")
#df8 = pd.read_csv("/content/batch_3_a9.csv")
#df9 = pd.read_csv("/content/batch_4_a11.csv")
#df10 = pd.read_csv("/content/batch_4_a9.csv")



df1_selected = df1[['question', 'answer']]
df2_selected = df2[['question', 'answer']]
df3_selected = df3[['question', 'answer']]
#df4_selected = df4[['question', 'answer']]
#df5_selected = df5[['question', 'answer']]
#df6_selected = df6[['question', 'answer']]
#df7_selected = df7[['question', 'answer']]
#df8_selected = df8[['question', 'answer']]
#df9_selected = df9[['question', 'answer']]
#df10_selected = df10[['question', 'answer']]


df=pd.concat([df1_selected,df2_selected,df3_selected], ignore_index=True)



In [ ]:
df = df.dropna()

In [ ]:
len(df)

4174

In [ ]:

questions = df["question"].tolist()
answers = df["answer"].tolist()

In [ ]:

tokenizer = T5Tokenizer.from_pretrained("t5-Base")
model = TFT5ForConditionalGeneration.from_pretrained("t5-Base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:

inputs = [f"question: {q} </s>" for q in questions]
outputs = [f"{a} </s>" for a in answers]

In [ ]:

input_encodings = tokenizer(inputs, padding=True, truncation=True, return_tensors="tf")
output_encodings = tokenizer(outputs, padding=True, truncation=True, return_tensors="tf")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


In [ ]:

input_ids = input_encodings['input_ids']
attention_mask = input_encodings['attention_mask']
labels = output_encodings['input_ids']

In [ ]:

train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels},
))

train_dataset = train_dataset.shuffle(1000).batch(16)


optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


def compute_loss(labels, logits):
    return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))


In [ ]:

model.compile(optimizer=optimizer, loss=compute_loss, metrics=['accuracy'])

model.fit(train_dataset, epochs=40)


Epoch 1/40
261/261 [==============================] - 437s 1s/step - loss: 0.7177 - accuracy: 0.8940
Epoch 2/40
261/261 [==============================] - 382s 1s/step - loss: 0.3512 - accuracy: 0.9279
Epoch 3/40
261/261 [==============================] - 383s 1s/step - loss: 0.3215 - accuracy: 0.9320
Epoch 4/40
261/261 [==============================] - 383s 1s/step - loss: 0.3006 - accuracy: 0.9348
Epoch 5/40
261/261 [==============================] - 383s 1s/step - loss: 0.2818 - accuracy: 0.9378
Epoch 6/40
261/261 [==============================] - 383s 1s/step - loss: 0.2658 - accuracy: 0.9402
Epoch 7/40
261/261 [==============================] - 383s 1s/step - loss: 0.2504 - accuracy: 0.9428
Epoch 8/40
261/261 [==============================] - 382s 1s/step - loss: 0.2362 - accuracy: 0.9453
Epoch 9/40
261/261 [==============================] - 383s 1s/step - loss: 0.2222 - accuracy: 0.9480
Epoch 10/40
261/261 [==============================] - 383s 1s/step - loss: 0.2102 - accura

In [ ]:
def generate_answer(question):
    input_text = f"question: {question} </s>"
    input_ids = tokenizer(input_text, return_tensors="tf").input_ids


    outputs = model.generate(
        input_ids,
        max_length=100,
        min_length=50,
        num_beams=5,
        do_sample=True,
        temperature=0.5,
        top_p=0.9
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer



# Exemple d’utilisation
print(generate_answer("What are good food during pregnancy"))

Foods rich in vitamins, minerals, and protein, such as lean meats, eggs, yogurt, salmon, avocados, edamame, nuts, kale, oats, eggs, kale, oats, and bananas, can support both your health and your baby's development.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_directory = "/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot/tokenizer_config.json',
 '/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot/special_tokens_map.json',
 '/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot/spiece.model',
 '/content/drive/MyDrive/fine_tuned_T5PregnancyChatbot/added_tokens.json')